In [1]:
import pytorch_lightning as pl
import pandas as pd
import os
import numpy as np

from src.misc.extracting_runs import extract_runs
import src.utils as utils
from src.data.dataloader import mtlDataModule
from src.pipeline.lightning_frame import mtlMayhemModule

MODEL_HPC = "/work3/s202821/models/"
WORKDIR_HPC = "/zhome/3b/d/154066/repos/multitask-mayhem/"
OUTDIR_HPC = "/work3/s202821/emp_models/"

WORKDIR_LOCAL = "/home/bnbj/repos/multitask-mayhem/"

os.chdir(WORKDIR_LOCAL)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
utils.set_seeds()

/home/bnbj/miniconda3/envs/multitask-mayhem/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Inference time

In [6]:
df_inf = pd.read_csv("reports/figures/inference_time.csv")
df_inf = df_inf.set_index("model")
styler = df_inf.style
styler.format("{:.2f}")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
styler.to_latex(
    buf=f"reports/figures/inf-time.tex",
    caption="Inference time comparison on NVIDIA A100-PCI GPU",
    label=f"tab:inf-time",
    position="htbp",
    position_float="centering",
    hrules=True,
    column_format="lccc",
    convert_css=True,
    )


## Separate runs on HPC

In [3]:
# extract_runs(workdir = MODEL_HPC, outdir = OUTDIR_HPC)

In [4]:
os.getcwd()

'/home/bbejczy/repos/multitask-mayhem'

## Run model on test set

In [24]:
df = pd.DataFrame(columns=["name","miou","map","person","forklift","agv","box"])

In [25]:
for root, folders, files in os.walk("reports/test_imgs/"):
    # print(folders)
    for file in files:
        if file.endswith(".csv"):
            # print(os.path.join(root, file))
            temp_df = pd.read_csv(os.path.join(root, file))
            df = pd.concat([df,temp_df], ignore_index=True)

In [26]:
df.sort_values(by=["map"], inplace=True, ascending=False)
# df["miou"] = (df["person"] + df["forklift"] + df["box"])/3
df

,name,miou,map,person,forklift,agv,box
1,frcnn-hybrid_dwa-10v1-graddrop_23-02-21T182345,0.872714,0.276550,0.306955,0.066649,0.500000,0.232595
31,frcnn-hybrid_5v1_23-02-16T001810,0.867079,0.203199,0.322933,0.059598,0.200000,0.230264
29,frcnn-hybrid_bmtl_23-02-16T003206,0.897177,0.194977,0.331060,0.081986,0.150000,0.216861
23,frcnn-hybrid_2v1_23-02-21T182529,0.880627,0.190339,0.309146,0.112894,0.100000,0.239316
6,frcnn-hybrid_relative_23-02-16T004645,0.887295,0.189558,0.324733,0.067644,0.166667,0.199187
3,frcnn-hybrid_dwa-graddrop_23-02-16T234854,0.899006,0.178334,0.284165,0.101613,0.100000,0.227559
33,frcnn-hybrid_dwa_23-02-16T234921,0.898418,0.175881,0.325917,0.096141,0.040000,0.241466
21,frcnn-hybrid_bmtl-graddrop_23-02-20T140735,0.881973,0.175497,0.292062,0.103664,0.083333,0.222928
11,frcnn-hybrid_10v1-graddrop_23-02-19T151850,0.875887,0.167123,0.328792,0.076890,0.071429,0.191382
18,frcnn-hybrid_uc_23-02-25T204205,0.872526,0.166600,0.367392,0.087119,0.003448,0.208442


In [27]:
df["map"] = (df["person"] + df["forklift"] + df["box"])/3
# try:
#     df.pop("agv")
# except:
#     pass
# df.pop("agv")
df = df.fillna(0)
df["name"] = df["name"]. apply(lambda x: x[:-16])
df["map"] = df["map"].apply(lambda x:  x*100)
df["person"] = df["person"].apply(lambda x:  x*100)
df["agv"] = df["agv"].apply(lambda x:  x*100)
df["forklift"] = df["forklift"].apply(lambda x:  x*100)
df["box"] = df["box"].apply(lambda x:  x*100)
df["miou"] = df["miou"].apply(lambda x:  x*100)
df.sort_values(by=["name"], inplace=True, ascending=False)
df.set_index("name", inplace=True)
df


styler = df.style

styler.format("{:.2f}")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
styler.to_latex(
    buf=f"reports/figures/test-set.tex",
    caption="Placeholder caption",
    label=f"tab:test-set",
    position="H",
    position_float="centering",
    hrules=True,
    column_format="lccccc",
    convert_css=True,
    )

In [6]:
import jinja2

jinja2.__version__

'2.11.3'

## Extract runs based on tags

In [2]:
import wandb
api = wandb.Api()

In [9]:
# TAG = "singlevsmulti"
# TAG = "weighting"
# TAG = "none-grad"
# TAG = "constant"
# TAG = "constant-grad"
# TAG = "uc-grad"
# TAG = "bmtl-grad"
# TAG = "dwa-grad"
# TAG = "control"

# tags_list = ["singlevsmulti","weighting","none-grad","constant","constant-grad","uc-grad","bmtl-grad","dwa-grad","control"]
tags_list = ["uc-grad"]

In [13]:
def print_tables(tags_list):

    df = pd.read_csv("reports/figures/runs_summary.csv")
    df.Tags = df.Tags.apply(lambda x: x.split(", "))

    df_control_run = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
    control_miou = 95.17
    control_map = 60.36
    df_control_run.Name = ["FRCNN-MTL"]
    df_control_run["mAP"] = [control_map]
    df_control_run["mIoU"] = [control_miou]
    df_control_run["delta mAP"] = [0]
    df_control_run["delta mIoU"] = [0]

    for tag in tags_list:
        print("Tag: ", tag)
        df.Output = df.Tags.apply(lambda x: tag in x)
        df_sampled = df[df.Output][["Name", "ID", "Tags"]]

        df_run_summary = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
        for (_, run_sample) in df_sampled.iterrows():
            print("Run: ", run_sample.Name[:-16])
            run = api.run("/thesis-mayhem/multitask-mayhem/runs/{}".format(run_sample.ID))
            df_run = run.history()
            summary_dict = {"Name": run_sample.Name[:-16], "mIoU": 0, "mAP": 0}

            try:
                summary_dict["mIoU"] = round(df_run["val_loss.seg"].max()*100, 2)
                summary_dict["delta mIoU"] = (summary_dict["mIoU"] - control_miou)
            except:
                pass

            try:
                summary_dict["mAP"] = round(df_run["val_loss.det"].max()*100, 2)
                summary_dict["delta mAP"] = (summary_dict["mAP"] - control_map)
            except:
                pass


            df_run_summary = df_run_summary.append(summary_dict, ignore_index=True)

        df_run_summary = pd.concat([df_control_run, df_run_summary], ignore_index=False)
        df_run_summary = df_run_summary.sort_values(by=["mAP", "mIoU"], ascending=False)
        df_run_summary.set_index("Name", inplace=True)

        styler = df_run_summary.style
        styler.format("{:.2f}")
        # styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
        # styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
        styler.to_latex(
            buf=f"reports/figures/{tag}.tex",
            caption="Placeholder caption",
            label=f"tab:{tag}",
            position="htbp",
            position_float="centering",
            hrules=True,
            column_format="lcccc",
            convert_css=True,
            )

In [14]:
print_tables(tags_list)

Tag:  uc-grad
Run:  frcnn-hybrid_uc-pcgrad
Run:  frcnn-hybrid_uc-cagrad
Run:  frcnn-hybrid_uc-graddrop
Run:  frcnn-hybrid_uc


In [333]:
df = pd.read_csv("reports/figures/runs_summary.csv")
df.Tags = df.Tags.apply(lambda x: x.split(", "))
df.Output = df.Tags.apply(lambda x: TAG in x)
df_sampled = df[df.Output][["Name", "ID", "Tags"]]
id_list = df_sampled.ID.tolist()
df_sampled

,Name,ID,Tags
5,frcnn-hybrid_bmtl-graddrop_23-02-20T140735,1hx7ni1c,[bmtl-grad]
7,frcnn-hybrid_bmtl-cagrad_23-02-20T132609,15wptbf7,[bmtl-grad]
8,frcnn-hybrid_bmtl-pcgrad_23-02-20T132549,148xg2lj,[bmtl-grad]
18,frcnn-hybrid_bmtl_23-02-16T003206,3ny1ktwq,"[bmtl-grad, bmtl-scale, weighting]"


In [345]:
df.control = df.Tags.apply(lambda x: "control" in x)
df_control = df[df.control][["Name", "ID"]]
df_control

,Name,ID,Tags
30,frcnn-hybrid_equal_23-02-14T235356,3g9znbyd,"[control, equal, none-grad, res, singlevsmulti..."


In [340]:
df.control = df.Tags.apply(lambda x: "control" in x)
df_control = df[df.control][["Name", "ID", "Tags"]]
run = api.run("/thesis-mayhem/multitask-mayhem/runs/{}".format(df_control.ID.item()))
control_miou = round(run.history()[["val_loss.seg"]].max().item()*100,2)
control_map = round(run.history()[["val_loss.det"]].max().item()*100, 2)
print(run.name)
print("Control mIoU: {}%".format(control_miou))
print("Control mAP: {}%".format(control_map))

# control_miou = 95.17
# control_map = 60.36
df_control_run = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
df_control_run.Name = [run.name[:-16]]
df_control_run["mAP"] = [control_map]
df_control_run["mIoU"] = [control_miou]
df_control_run["delta mAP"] = [0]
df_control_run["delta mIoU"] = [0]

frcnn-hybrid_equal_23-02-14T235356
Control mIoU: 94.03%
Control mAP: 60.36%


In [341]:
df_control_run

,Name,mAP,mIoU,delta mAP,delta mIoU
0,frcnn-hybrid_equal,60.36,94.03,0,0


In [335]:
for (header, run_sample) in df_sampled.iterrows():
    print(run_sample.ID)

1hx7ni1c
15wptbf7
148xg2lj
3ny1ktwq


In [336]:
df_run_summary = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
for (_, run_sample) in df_sampled.iterrows():
    run = api.run("/thesis-mayhem/multitask-mayhem/runs/{}".format(run_sample.ID))
    df_run = run.history()
    summary_dict = {"Name": run_sample.Name[:-16], "mIoU": 0, "mAP": 0}

    try:
        summary_dict["mIoU"] = round(df_run["val_loss.seg"].max()*100, 2)
        summary_dict["delta mIoU"] = (summary_dict["mIoU"] - control_miou)
    except:
        pass

    try:
        summary_dict["mAP"] = round(df_run["val_loss.det"].max()*100, 2)
        summary_dict["delta mAP"] = (summary_dict["mAP"] - control_map)
    except:
        pass


    df_run_summary = df_run_summary.append(summary_dict, ignore_index=True)

In [337]:
df_run_summary = pd.concat([df_control_run, df_run_summary], ignore_index=False)
df_run_summary = df_run_summary.sort_values(by=["mAP", "mIoU"], ascending=False)
df_run_summary.set_index("Name", inplace=True)


In [338]:
styler = df_run_summary.style
styler.format("{:.2f}")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
styler.to_latex(
    buf=f"reports/figures/{TAG}.tex",
    caption="Placeholder caption",
    label=f"tab:{TAG}",
    position="htbp",
    position_float="centering",
    hrules=True,
    column_format="lcccc",
    convert_css=True,
    )


# Demo inference metrics

In [40]:
df_demo = pd.read_csv("reports/figures/23-02-16T185002")
df_demo = df_demo[1:]
df_demo
# df_demo["pass"].mean()

,prep,pass,vis,call
1,5.702624,206.781082,19.543615,317.054871
2,4.795840,203.308441,10.331456,360.043274
3,3.763040,202.379486,10.378688,324.874146
4,4.475968,204.806747,10.770080,378.795105
5,5.036672,219.207550,11.212160,323.905731
...,...,...,...,...
304,4.754016,206.410843,20.698849,278.020325
305,3.723584,198.522598,18.364639,277.834076
306,4.306624,197.821213,22.708160,259.826202
307,3.828640,201.499359,23.370848,270.333832


In [38]:
df_demo["pass"].std()

14.035310167587634